In [1]:
!pip install openai --upgrade
!pip install pytelegrambotapi --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 5.2 MB/s eta 0:00:00


In [2]:
words = ["reassure",
"devote life to",
"worthwhile",
"a good influence on",
"be an honor",
"make a contribution",
"take pleasure in",
"beneficial",
"be use of",
"make a difference",
"get more satisfaction out of",
"foolish",
"have appeal",
"practical",
"purchase",
"have potential",
"convince",
"ridiculous",
"regret",
"encourage",
"make financial sense",
"urge",
"disorganized",
"hang up",
"line up",
"tangle up",
"arrange",
"neat",
"all over the place",
"jumbled up",
"throw on",
"fold",
"put in a pile",
"put away",
"thoroughly",
"at own pace",
"on own time",
"little by little",
"smoothly",
"steadily",
"expected",
"successfully",
"effectively",
"efficiently",
"with ease",
"to cause",
"look for",
"fortunate",
"coincidence",
"fate",
"belive my luck",
"be in the right place in the right time",
"deliberate decision",
"life-changing experience",
"path",
"chance encounter",
"determination",
"lucky break",
"wind up",
"incompetent",
"learn the hard way",
"be dumb thing to do",
"be in too much of a hurry",
"find yourself in an awkward situation",
"be a silly mistake",
"to kick",
"see the funny side",
"clothes",
"education",
"humor",
"goals",
"personality",
"hairstyle",
"analyze",
"examine",
"assess",
"identify",
"survey",
"calculate",
"authentic",
"genuine",
"original",
"counterheit",
"illegal",
"second-rate",
"deadly",
"imperfect",
"sophisticated",
"fake",
"inferior",
"accurate",
"false",
"rumor",
"biased",
"hoax",
"suspicious",
"controversial",
"trustworthy",
"dishonest",
"misinformation",
"urban legend",
"exaggerated",
"misleading",
"white lie",
"analytical",
"intellectual",
"artistic",
"logical",
"athletic",
"musical",
"competent",
"skilled",
"determined",
"talented",
"gifted",
"technical",
"imaginative",
"trained",
"be a downer",
"brighten up",
"put mind at rest",
"stress out",
"leave a lasting impression on",
"raise spirits",
"take mind off",
"certain",
"confident",
"determined",
"guarantee",
"harm",
"planned",
"reason",
"scenario",
"ultimate",
"character",
"performance",
"difficulties",
"photograph",
"encouragement",
"photography",
"interviewer"]

In [10]:
from openai import OpenAI
import telebot
import sys
import time
import os
import sqlite3
import json
from telebot import types

conn = sqlite3.connect('telegram_bot.db', check_same_thread=False)
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY,
    messages TEXT
)''')
conn.commit()

cursor.execute("PRAGMA table_info(users)")
columns = [column[1] for column in cursor.fetchall()]
if 'messages' not in columns:
    cursor.execute("ALTER TABLE users ADD COLUMN messages TEXT DEFAULT '[]'")
    conn.commit()

cursor.execute("UPDATE users SET messages = '[]' WHERE messages IS NULL")
conn.commit()

text_base = f"Answer only in English. You are a game provider whose main aim is to take one random word from list {words} and create an interesting riddle about this word. Do not text this word, just create a riddle. Wait for user's answer, when user answers correctly, you should gave him 10 social points (remember their amount). If user gave a wrong answer try to help him softly if he is wrong twice in a row, subtract 20 social points. User starts from 0 points. If user is right ask him if he wants to continue playing and do it if he wants."

bot = telebot.TeleBot("6538393447:AAEEjPYro0kEnY8KdntvdM8_GINraSPN9tI")
c = OpenAI(base_url="https://api.tensorai.ru/v1/openai", api_key="f45ce354914e3b1257cdd2c91526d3")

def get_user_messages(user_id):
    cursor.execute('SELECT messages FROM users WHERE user_id = ?', (user_id,))
    row = cursor.fetchone()
    if row is None:
        cursor.execute('INSERT INTO users (user_id, messages) VALUES (?, ?)', (user_id, json.dumps([])))
        conn.commit()
        return []
    return json.loads(row[0])

def save_user_messages(user_id, messages):
    cursor.execute('UPDATE users SET messages = ? WHERE user_id = ?', (json.dumps(messages), user_id))
    conn.commit()

def print_res(user_input, user_id):
    messages = get_user_messages(user_id)
    messages.append({"role": "user", "content": user_input})
    try:
        resp = c.chat.completions.create(
            max_tokens=1000,
            model="llama_3_70b",
            messages=[
                {"role": "system", "content": text_base},
                *messages
            ]
        )
        choice = resp.choices[0]
        message_content = choice.message.content
        messages.append({"role": "assistant", "content": message_content})
        save_user_messages(user_id, messages)
        return message_content
    except Exception as e:
        return str(e)

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    user_id = message.from_user.id
    bot.reply_to(message, "Hello, I'm your English game provider! Let's start a new game!")

@bot.message_handler(func=lambda message: True)
def echo_all(message):
    user_id = message.from_user.id
    bot.reply_to(message, print_res(message.text, user_id))

@bot.message_handler(commands=['restart'])
def send_restart(message):
    markup = types.InlineKeyboardMarkup()
    restart_button = types.InlineKeyboardButton(text="Перезапустить бота", callback_data="restart")
    markup.add(restart_button)
    bot.send_message(message.chat.id, "Нажмите на кнопку для перезапуска бота.", reply_markup=markup)

@bot.callback_query_handler(func=lambda call: True)
def handle_query(call):
    if call.data == "restart":
        bot.answer_callback_query(call.id, "Бот перезапускается...")
        bot.stop_polling()
        time.sleep(5)
        os.execv(sys.executable, ['python'] + sys.argv)

bot.infinity_polling()

2024-05-22 06:00:17,186 (__init__.py:1101 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2024-05-22 06:00:17,190 (__init__.py:1103 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
